In [9]:
from enhance_ocod.labelling.ner_spans import lfs
from enhance_ocod.labelling.weak_labelling import (process_dataframe_batch, create_flat_tag, 
create_commercial_park_tag, remove_overlapping_spans)
import pandas as pd 




In [2]:
df = pd.read_csv('data/enhanced_ocod_data_and_gt/ground_truth_dev_set_labels.csv')
df = df['property_address'].drop_duplicates().reset_index(drop=True).to_frame()


df= create_flat_tag(df)

df = create_commercial_park_tag(df)

/home/jonno/enhance_ocod/src/enhance_ocod/labelling/weak_labelling.py:204: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['flat_tag'] = df[text_column].str.contains(
/home/jonno/enhance_ocod/src/enhance_ocod/labelling/weak_labelling.py:212: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['commercial_park_tag'] = df[text_column].str.contains(


In [3]:
df

,property_address,flat_tag,commercial_park_tag
0,"207 sloane street, london (sw1x 9qx)",False,False
1,"apartment 533, block 11 spectrum, blackfriars ...",True,False
2,"flat 9, st. james heights, paradise walk, bexh...",True,False
3,"land at the side of greyhound service station,...",False,False
4,"land and buildings at brook road, bootle",False,False
...,...,...,...
1993,"flat 542, nell gwynn house, sloane avenue, lon...",True,False
1994,300 to 324 (even) spen lane and 1 - 17 (odd) 2...,False,False
1995,"highgate mills, highgate road, queensbury, bra...",False,False
1996,land and buildings on the east side of falcon ...,False,False


In [17]:
test = process_dataframe_batch(df, 
                           batch_size = 5000,
                           text_column = 'property_address',
                           include_function_name = False,
                           save_intermediate = False,
                           output_dir = None,
                           verbose = True)

Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]/home/jonno/enhance_ocod/src/enhance_ocod/labelling/weak_labelling.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['text'] = batch_df[text_column]
Processing batches: 100%|██████████| 1/1 [00:01<00:00,  1.20s/it]


Processing complete:
  - Processed 1998 rows successfully
  - Found 10094 total spans
  - Average 5.05 spans per row


In [5]:
test

[{'text': '207 sloane street, london (sw1x 9qx)',
  'spans': [{'text': 'london', 'start': 19, 'end': 25, 'label': 'CITY'},
   {'text': 'sw1x 9qx', 'start': 27, 'end': 35, 'label': 'POSTCODE'},
   {'text': 'sloane street', 'start': 4, 'end': 17, 'label': 'STREET'},
   {'text': '207', 'start': 0, 'end': 3, 'label': 'STREET_NUMBER'}],
  'row_id': 0},
 {'text': 'apartment 533, block 11 spectrum, blackfriars road, salford (m3 7ee)',
  'spans': [{'text': 'salford', 'start': 52, 'end': 59, 'label': 'CITY'},
   {'text': 'm3 7ee', 'start': 61, 'end': 67, 'label': 'POSTCODE'},
   {'text': 'blackfriars road', 'start': 34, 'end': 50, 'label': 'STREET'},
   {'text': '533', 'start': 10, 'end': 13, 'label': 'UNIT_ID'},
   {'text': 'apartment', 'start': 0, 'end': 9, 'label': 'UNIT_TYPE'}],
  'row_id': 1},
 {'text': 'flat 9, st. james heights, paradise walk, bexhill-on-sea (tn40 2lg)',
  'spans': [{'text': 'st. james heights',
    'start': 8,
    'end': 25,
    'label': 'BUILDING'},
   {'text': 'bexhil

In [22]:
remove_overlapping_spans(test)

In [23]:
get_overlap_stats(test)

{'total_rows': 1998,
 'rows_with_overlaps': 0,
 'overlap_percentage': 0.0,
 'total_spans_before': 8471,
 'total_overlaps_removed': 0,
 'spans_after': 8471}

In [24]:
test

[{'text': '207 sloane street, london (sw1x 9qx)',
  'spans': [{'text': '207', 'start': 0, 'end': 3, 'label': 'STREET_NUMBER'},
   {'text': 'sloane street', 'start': 4, 'end': 17, 'label': 'STREET'},
   {'text': 'london', 'start': 19, 'end': 25, 'label': 'CITY'},
   {'text': 'sw1x 9qx', 'start': 27, 'end': 35, 'label': 'POSTCODE'}],
  'row_id': 0},
 {'text': 'apartment 533, block 11 spectrum, blackfriars road, salford (m3 7ee)',
  'spans': [{'text': 'apartment', 'start': 0, 'end': 9, 'label': 'UNIT_TYPE'},
   {'text': '533', 'start': 10, 'end': 13, 'label': 'UNIT_ID'},
   {'text': 'blackfriars road', 'start': 34, 'end': 50, 'label': 'STREET'},
   {'text': 'salford', 'start': 52, 'end': 59, 'label': 'CITY'},
   {'text': 'm3 7ee', 'start': 61, 'end': 67, 'label': 'POSTCODE'}],
  'row_id': 1},
 {'text': 'flat 9, st. james heights, paradise walk, bexhill-on-sea (tn40 2lg)',
  'spans': [{'text': 'flat', 'start': 0, 'end': 4, 'label': 'UNIT_TYPE'},
   {'text': '9', 'start': 5, 'end': 6, 'labe

In [15]:
from typing import List, Dict, Any, Callable, Optional, Union

# More memory-efficient version for very large datasets
def remove_overlapping_spans(results: List[Dict[str, Any]]) -> None:
    """
    In-place version that modifies the input list directly.
    More memory efficient for very large datasets.
    """
    
    for result in results:
        spans = result['spans']
        
        if len(spans) <= 1:
            continue
        
        # Sort by length descending
        spans.sort(key=lambda x: x['end'] - x['start'], reverse=True)
        
        kept_spans = []
        
        for span in spans:
            # Check overlap with kept spans
            if not any(not (span['end'] <= kept['start'] or kept['end'] <= span['start']) 
                      for kept in kept_spans):
                kept_spans.append(span)
        
        # Sort by start position
        kept_spans.sort(key=lambda x: x['start'])
        result['spans'] = kept_spans

# Batch processing function with overlap removal
def process_and_clean_batch(df: pd.DataFrame, 
                           remove_overlaps: bool = True,
                           **kwargs) -> List[Dict[str, Any]]:
    """
    Process dataframe in batches and remove overlaps in one step.
    Most efficient for large datasets.
    """
    
    # Process the batch first
    results = process_dataframe_batch(df, **kwargs)
    
    # Remove overlaps if requested
    if remove_overlaps:
        remove_overlapping_spans_inplace(results)
    
    return results

# Example usage functions optimized for your datasets
def process_training_set_clean(df: pd.DataFrame, 
                              output_dir: str = './training_intermediate') -> List[Dict[str, Any]]:
    """Process 100k training set with overlap removal"""
    
    return process_and_clean_batch(
        df,
        batch_size=5000,
        save_intermediate=True,
        output_dir=output_dir,
        remove_overlaps=True,
        verbose=True
    )

# Utility function to get overlap statistics (useful for debugging)
def get_overlap_stats(results: List[Dict[str, Any]]) -> Dict[str, Any]:
    """Get statistics about overlaps before removal (for debugging)"""
    
    total_rows = len(results)
    rows_with_overlaps = 0
    total_spans_before = 0
    total_overlaps_removed = 0
    
    for result in results:
        spans = result['spans'][:]  # Copy to avoid modifying original
        total_spans_before += len(spans)
        
        if len(spans) <= 1:
            continue
            
        # Check for overlaps
        spans.sort(key=lambda x: x['start'])
        has_overlap = False
        
        for i in range(len(spans) - 1):
            if spans[i]['end'] > spans[i + 1]['start']:
                has_overlap = True
                break
        
        if has_overlap:
            rows_with_overlaps += 1
            
            # Count overlaps by applying removal algorithm
            spans.sort(key=lambda x: x['end'] - x['start'], reverse=True)
            kept = []
            
            for span in spans:
                if not any(not (span['end'] <= k['start'] or k['end'] <= span['start']) 
                          for k in kept):
                    kept.append(span)
            
            total_overlaps_removed += len(spans) - len(kept)
    
    return {
        'total_rows': total_rows,
        'rows_with_overlaps': rows_with_overlaps,
        'overlap_percentage': (rows_with_overlaps / total_rows) * 100,
        'total_spans_before': total_spans_before,
        'total_overlaps_removed': total_overlaps_removed,
        'spans_after': total_spans_before - total_overlaps_removed
    }